<a href="https://colab.research.google.com/github/OdysseusPolymetis/digital_classics_course/blob/main/5_scansion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cltk

In [ ]:
from cltk.prosody.lat.scanner import Scansion

In [ ]:
Scansion().scan_text('dedērunt te miror antōnī quorum. sī quid est in mē ingenī jūdicēs quod sentiō.')

['u--uuu---ux', 'u---u--u---ux']

In [ ]:
from cltk.prosody.grc import Scansion

In [ ]:
texte = """
Ἄνδρα μοι ἔννεπε, Μοῦσα, πολύτροπον, ὃς μάλα πολλὰ
πλάγχθη, ἐπεὶ Τροίης ἱερὸν πτολίεθρον ἔπερσε,
πολλῶν δ´ ἀνθρώπων ἴδεν ἄστεα καὶ νόον ἔγνω,
πολλὰ δ´ ὅ γ´ ἐν πόντῳ πάθεν ἄλγεα ὃν κατὰ θυμόν,
ἀρνύμενος ἥν τε ψυχὴν καὶ νόστον ἑταίρων.
ἀλλ´ οὐδ´ ὧς ἑτάρους ἐρρύσατο, ἱέμενός περ.
"""

scanner = Scansion()

verses = texte.strip().split('\n')

resultats_scansion = []

for vers in verses:
    scanned = scanner.scan_text(vers+".")
    resultats_scansion.append(scanned)

for i, resultat in enumerate(resultats_scansion):
    print(f"Vers {i+1}: {resultat}")

INFO:CLTK:IndexError while checking if syllable 'λλα' is long. Continuing.
INFO:CLTK:IndexError while checking if syllable 'ρσε' is long. Continuing.
INFO:CLTK:IndexError while making syllables of 'δ´'. Continuing.
INFO:CLTK:IndexError while checking if syllable 'γνω' is long. Continuing.
INFO:CLTK:IndexError while making syllables of 'δ´'. Continuing.
INFO:CLTK:IndexError while making syllables of 'γ´'. Continuing.
INFO:CLTK:IndexError while checking if syllable 'θυμόν' is long. Continuing.
INFO:CLTK:IndexError while checking if syllable 'ίρων' is long. Continuing.
INFO:CLTK:IndexError while checking if syllable 'περ' is long. Continuing.


Vers 1: ['¯˘¯¯˘˘¯˘˘˘˘¯˘˘x']
Vers 2: ['¯˘¯˘¯˘˘¯˘˘˘˘¯x']
Vers 3: ['˘¯¯¯˘˘¯˘¯¯˘˘x']
Vers 4: ['˘˘˘¯¯˘¯˘¯¯˘˘x']
Vers 5: ['¯˘˘¯¯˘¯¯˘˘˘x']
Vers 6: ['˘˘¯˘˘˘˘˘˘¯x']


In [ ]:
!git clone https://github.com/cltk/grc_text_tesserae.git

Cloning into 'grc_text_tesserae'...
remote: Enumerating objects: 1235, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 1235 (delta 0), reused 0 (delta 0), pack-reused 1217
Receiving objects: 100% (1235/1235), 34.61 MiB | 17.74 MiB/s, done.
Resolving deltas: 100% (365/365), done.
Updating files: 100% (826/826), done.


In [ ]:
import os
import re
from collections import Counter
import logging

logging.basicConfig(level=logging.WARNING)

scanner = Scansion()

dossier = "/content/grc_text_tesserae/texts"
fichiers = [f for f in os.listdir(dossier) if f.endswith('.tess') and "homer" in f.lower()]

scansions = []

for fichier in fichiers:
    with open(os.path.join(dossier, fichier), 'r', encoding='utf-8') as f:
        for ligne in f:
            vers_propre = re.sub(r'<[^>]+>', '', ligne).strip() + '.'

            try:
                scanned_vers = scanner.scan_text(vers_propre)
                scansions.extend(scanned_vers)
            except Exception as e:
                print(f"Erreur de scansion pour le vers : {vers_propre}")
                print(e)

scansion_counter = Counter(scansions)
cinq_plus_frequents = scansion_counter.most_common(5)

for scansion, freq in cinq_plus_frequents:
    print(f"Scansion: {scansion}, Fréquence: {freq}")


In [ ]:
list_of_sentences=['dedērunt te miror antōnī quorum. sī quid est in mē ingenī jūdicēs quod sentiō.', 'dedērunt te miror antōnī quorum. sī quid est in mē ingenī jūdicēs quod sentiō.']

In [ ]:
len(list_of_sentences)

2

In [ ]:
for phrase in list_of_sentences:
  result=Scansion().scan_text(phrase)
  print(result)

['u--uuu---ux', 'u---u--u---ux']
['u--uuu---ux', 'u---u--u---ux']
